In [3]:
import torch
import torchvision

ModuleNotFoundError: No module named 'torchvision'

In [4]:
!pip install torchvision

   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   -- ------------------------------------- 0.1/1.2 MB 1.7 MB/s eta 0:00:01
   ---- ----------------------------------- 0.1/1.2 MB 2.1 MB/s eta 0:00:01
   ----- ---------------------------------- 0.2/1.2 MB 1.3 MB/s eta 0:00:01
   ----- ---------------------------------- 0.2/1.2 MB 1.3 MB/s eta 0:00:01
   ----- ---------------------------------- 0.2/1.2 MB 1.3 MB/s eta 0:00:01
   ------ --------------------------------- 0.2/1.2 MB 831.5 kB/s eta 0:00:02
   ------- -------------------------------- 0.2/1.2 MB 811.5 kB/s eta 0:00:02
   ------- -------------------------------- 0.2/1.2 MB 811.5 kB/s eta 0:00:02
   ------- -------------------------------- 0.2/1.2 MB 686.8 kB/s eta 0:00:02
   --------- ------------------------------ 0.3/1.2 MB 681.0 kB/s eta 0:00:02
   --------- ------------------------------ 0.3/1.2 MB 681.0 kB/s eta 0:00:02
   ---------- ----------------------------- 0.3/1.2 MB 656.4 kB/s eta 0:00:02
   --

  You can safely remove it manually.


   -------- ------------------------------ 34.6/159.8 MB 424.0 kB/s eta 0:04:56
   -------- ------------------------------ 34.6/159.8 MB 423.2 kB/s eta 0:04:56
   -------- ------------------------------ 34.6/159.8 MB 423.5 kB/s eta 0:04:56
   -------- ------------------------------ 34.6/159.8 MB 422.9 kB/s eta 0:04:56
   -------- ------------------------------ 34.7/159.8 MB 423.5 kB/s eta 0:04:56
   -------- ------------------------------ 34.7/159.8 MB 423.2 kB/s eta 0:04:56
   -------- ------------------------------ 34.7/159.8 MB 422.7 kB/s eta 0:04:56
   -------- ------------------------------ 34.7/159.8 MB 423.8 kB/s eta 0:04:56
   -------- ------------------------------ 34.7/159.8 MB 423.2 kB/s eta 0:04:56
   -------- ------------------------------ 34.8/159.8 MB 424.0 kB/s eta 0:04:55
   -------- ------------------------------ 34.8/159.8 MB 423.8 kB/s eta 0:04:56
   -------- ------------------------------ 34.8/159.8 MB 424.6 kB/s eta 0:04:55
   -------- ----------------------------

In [5]:
!python run.py 

c:\Users\21520\anaconda3\.conda\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\21520\anaconda3\.conda\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Traceback (most recent call last):
  File "e:\msrlOB-1\run.py", line 434, in <module>
    demo_single_image(args, args.image_name)
  File "e:\msrlOB-1\run.py", line 360, in demo_single_image
    image_original = Image.open(image_path)
                     ^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\21520\anaconda3\.conda\Lib\site-packages\PIL\Image.py", line 3277, 